In [23]:
import trimet_plan_game_helper_scripts as helpers
import geopandas as gpd
import boto3
from shapely.geometry import LineString
import pandas as pd
import seaborn as sns

In [2]:
tm_boundary = gpd.read_file("gis/export/tm_route_buffer_bounds.geojson")
trimet_crs = "EPSG:2913"

In [3]:
gdf_points, itineraries_reduced, itinerary_routes_reduced, tries = helpers.generate_random_points_make_itinerary(tm_boundary, trimet_crs)

In [6]:
itinerary_routes_reduced

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_id_combo
1,0,4785,1426,2352,1007,1996.076433,1,24,BUS,7169,NW 23rd & Thurman,1846,NE Fremont & M L King,"LINESTRING (-122.69886 45.53575, -122.69888 45...",24 to 6
3,0,4785,1426,2352,1007,1996.076433,3,6,BUS,5910,NE M L King & Fremont,1026,Jantzen Beach Main Stop,"LINESTRING (-122.66153 45.54816, -122.66153 45...",24 to 6
6,1,5112,1508,3205,399,2107.343015,1,77,BUS,14032,NW Thurman & 23rd,4042,NE Multnomah & Grand,"LINESTRING (-122.69836 45.53550, -122.69680 45...",77 to 6


In [10]:
itinerary_routes_reduced['route_id'] = itinerary_routes_reduced['route_id'].astype(int)

In [5]:
gtfs_routes = pd.read_csv("gtfs/routes.txt")
gtfs_routes.head(2)

,route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color,route_sort_order
0,1,TRIMET,1,Vermont,3,https://trimet.org/schedules/r001.htm,NaN,NaN,400
1,2,TRIMET,FX2,Division,3,https://trimet.org/schedules/r002.htm,61A60E,FFFFFF,500


In [13]:
itinerary_routes_long_name = itinerary_routes_reduced.merge(gtfs_routes[['route_id','route_long_name']], how='inner', on='route_id')

,itin_idx,totalTime,walkTime,transitTime,waitingTime,walkDistance,leg_id,route_id,mode,fromStopCode,fromName,toStopCode,toName,legGeometry,route_id_combo,route_long_name
0,0,4785,1426,2352,1007,1996.076433,1,24,BUS,7169,NW 23rd & Thurman,1846,NE Fremont & M L King,"LINESTRING (-122.69886 45.53575, -122.69888 45...",24 to 6,Fremont/NW 18th
1,0,4785,1426,2352,1007,1996.076433,3,6,BUS,5910,NE M L King & Fremont,1026,Jantzen Beach Main Stop,"LINESTRING (-122.66153 45.54816, -122.66153 45...",24 to 6,Martin Luther King Jr Blvd
2,1,5112,1508,3205,399,2107.343015,1,77,BUS,14032,NW Thurman & 23rd,4042,NE Multnomah & Grand,"LINESTRING (-122.69836 45.53550, -122.69680 45...",77 to 6,Broadway/Halsey


In [24]:
# Get Unique continents
color_labels = itinerary_routes_long_name['route_id'].unique()
# List of colors in the color palettes
hex_values = sns.color_palette("Set2", len(color_labels)).as_hex()
# Map continents to the colors
color_map = dict(zip(color_labels, hex_values))

In [25]:
itinerary_routes_long_name['route_color'] = itinerary_routes_long_name['route_id'].apply(lambda x: color_map[x])

In [26]:
itinerary_routes_long_name['dropdown_route'] = itinerary_routes_long_name.apply(lambda x: str(x['route_id'])+" - "+str(x['route_long_name']), axis=1)

In [27]:
gdf_points.to_file("gis/export/origin_destination_points.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/origin_destination_points.geojson", "meysohn-sandbox", "trimet_trip_planner/origin_destination_points.geojson",ExtraArgs={'ACL':'public-read'})

In [28]:
itineraries_reduced.to_file("gis/export/itineraries_reduced.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/itineraries_reduced.geojson", "meysohn-sandbox", "trimet_trip_planner/itineraries_reduced.geojson",ExtraArgs={'ACL':'public-read'})

In [29]:
itinerary_routes_long_name.to_file("gis/export/itinerary_routes_long_name.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/itinerary_routes_long_name.geojson", "meysohn-sandbox", "trimet_trip_planner/itinerary_routes_long_name.geojson",ExtraArgs={'ACL':'public-read'})

In [20]:
origin_destination_centriod = LineString(list(gdf_points['points'].to_numpy())).centroid

In [21]:
origin_destination_centriod_gdf = gpd.GeoDataFrame(pd.DataFrame(['origin_dest_centroid'], columns=['id']),crs="EPSG:4326", geometry=[origin_destination_centriod])
origin_destination_centriod_gdf

,id,geometry
0,origin_dest_centroid,POINT (-122.69319 45.57985)


In [22]:
origin_destination_centriod_gdf.to_file("gis/export/origin_destination_centriod.geojson", driver="GeoJSON")
client = boto3.client("s3")
client.upload_file("gis/export/origin_destination_centriod.geojson", "meysohn-sandbox", "trimet_trip_planner/origin_destination_centriod.geojson",ExtraArgs={'ACL':'public-read'})